In [1]:
%config Completer.use_jedi = False

In [2]:
from time import time
import torch
from torch import nn
import torch.optim as optim
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import norm

# Forward KL vs Reverse KL vs JSD

In [3]:
def get_gaussian(mu, sigma):
    d = torch.distributions.normal.Normal(mu, sigma)
    return lambda X: d.log_prob(X)

In [4]:
p = get_gaussian(0, 1)

In [5]:
X = torch.from_numpy(np.linspace(-10,10,500)).double()

In [21]:
plt.plot(X.numpy(), p(X).exp().numpy())

In [7]:
def pi(x):
    # define a mixture of gaussians
    k = 0.8
    a = get_gaussian(-2, 0.5)
    b = get_gaussian(2, 0.5)
    
    # your code here
    pass

In [8]:
plt.plot(X.numpy(), pi(X).exp().numpy())

In [9]:
def plot_distr(p,pi,X):
    with torch.no_grad():
        res1 = p(X).exp()
        res2 = pi(X).exp()

        plt.plot(X.numpy(), res1.numpy(), 'g--', label=r'$p(x)$')
        plt.plot(X.numpy(), res2.numpy(), label=r'$\pi(x)$')
        plt.legend()
        plt.show()

In [9]:
plot_distr(p, pi, X)

<img src="pics/kld_jsd.jpg" width=800 height=800 />

In [12]:
def KLD(q,p,X):
    # p and q return log pdf
    # X - some linspace
    pass

In [13]:
def JSD(q,p,X):
    return (KLD(q,p,X) + KLD(p,q,X)) / 2

In [11]:
mu_start = -1
sigma_start = 0.5

In [12]:
# define trainable params
mu = torch.tensor(mu_start, requires_grad=True, dtype=torch.double)
sigma = torch.tensor(sigma_start, requires_grad=True, dtype=torch.double)
p = get_gaussian(mu, sigma)

plot_distr(p, pi, X)

In [33]:
learning_rate = 1e-1

In [34]:
optimizer = torch.optim.SGD([mu, sigma], lr=learning_rate)

In [13]:
for i in range(10):
    # write the train loop for forward KL
    print(i)
    optimizer.zero_grad()
    
    # your code here
    
    plot_distr(p, pi, X)

In [19]:
forward_KL_dist = [mu.detach().numpy(), sigma.detach().numpy()]

In [ ]:
# your code here

In [23]:
reverse_KLD_dist = [mu.detach().numpy(), sigma.detach().numpy()]

In [ ]:
# your code here

In [ ]:
JSD_dist = [mu.detach().numpy(), sigma.detach().numpy()]

In [28]:
# plot all distr

# Vanilla GAN

<img src="pics/gan_objective.jpg" width=800 height=800 />

In [29]:
def get_simple_model(hiddens):
    assert len(hiddens) > 1

    modules = []
    for in_, out_ in zip(hiddens[:-2], hiddens[1:-1]):
        modules.extend([nn.Linear(in_, out_), nn.ReLU()])

    modules.append(nn.Linear(hiddens[-2], hiddens[-1]))

    return nn.Sequential(*modules)

In [30]:
# define shallow nn
gen_hiddens = [1,64,64,1]
dis_hiddens = [1,64,64,1]

In [ ]:
####

In [49]:
mu = 2
noise_fn = lambda x: torch.rand((x, 1), device='cpu')-2
data_fn = lambda x: mu+torch.randn((x, 1), device='cpu')
data_pdf = lambda X: norm.pdf(X-mu)

In [50]:
def plot_gan_data(data_fn, noise_fn, data_pdf=None):
    noise = noise_fn(5000).numpy().flatten()
    target = data_fn(5000).numpy().flatten()
    
    plt.hist(noise, label='noise', alpha=0.5, density=True, color='b')
    plt.hist(target, label='target', alpha=0.5, density=True, color='g')
    if data_pdf is not None:
        x = np.linspace(-6,6,100)
        plt.plot(x, data_pdf(x), 'g', label='real distibution')

    plt.legend(loc='upper left')
    plt.show()

In [16]:
plot_gan_data(data_fn, noise_fn, data_pdf)

In [52]:
class VanillaGAN():
    def __init__(self, G, D, noise_fn, data_fn,
                 batch_size=32, device='cpu', lr_D=1e-3, lr_G=2e-4):
        """A GAN class for holding and training a generator and discriminator
        Args:
            G: a Ganerator network
            D: A Discriminator network
            noise_fn: function f(num: int) -> pytorch tensor, (latent vectors)
            data_fn: function f(num: int) -> pytorch tensor, (real samples)
            batch_size: training batch size
            device: cpu or CUDA
            lr_D: learning rate for the discriminator
            lr_G: learning rate for the generator
        """
        self.G = G
        self.G = self.G.to(device)
        self.D = D
        self.D = self.D.to(device)
        self.noise_fn = noise_fn
        self.data_fn = data_fn
        self.batch_size = batch_size
        self.device = device
        # !
        self.criterion = nn.BCELoss()
        self.optim_D = optim.Adam(D.parameters(),
                                  lr=lr_D, betas=(0.5, 0.999))
        self.optim_G = optim.Adam(G.parameters(),
                                  lr=lr_G, betas=(0.5, 0.999))
        # is needed in D train loop
        self.target_ones = torch.ones((batch_size, 1)).to(device)
        self.target_zeros = torch.zeros((batch_size, 1)).to(device)
    
    def generate_samples(self, latent_vec=None, num=None):
        """Sample from the generator.
        Args:
            latent_vec: A pytorch latent vector or None
            num: The number of samples to generate if latent_vec is None
        If latent_vec and num are None then us self.batch_size random latent
        vectors.
        ! We don't need grad for generated samples
        """
        num = self.batch_size if num is None else num
        latent_vec = self.noise_fn(num) if latent_vec is None else latent_vec
        # your code here
        samples = []
        
        return samples

    def train_step_G(self):
        """Train the generator one step and return the loss."""
        self.G.zero_grad()
        latent_vec = self.noise_fn(self.batch_size)
        # your code here
        # use self.target_ones
        loss = 0
        
        return loss

    def train_step_D(self):
        """Train the discriminator one step and return the losses."""
        self.D.zero_grad()

        # real samples
        real_samples = self.data_fn(self.batch_size)
        # calc real loss
        # you code here
        loss_real = 0

        # generated samples
        latent_vec = self.noise_fn(self.batch_size)
        # calc fake loss
        # you shouldn't optimize G here
        # you code here
        
        loss_fake = 0

        # combine
        loss = (loss_real + loss_fake) / 2
        loss.backward()
        self.optim_D.step()
        return loss_real, loss_fake

    def train_step(self):
        """Train both networks and return the losses."""
        loss_D = self.train_step_D()
        loss_G = self.train_step_G()
        return loss_G, loss_D

In [53]:
gen_hiddens = [1,64,64,1]
dis_hiddens = [1,64,64,1]
G = get_simple_model(gen_hiddens)
D = nn.Sequential(*get_simple_model(dis_hiddens), nn.Sigmoid())

gan = VanillaGAN(G, D, noise_fn, data_fn, device='cpu')

In [54]:
def visualize_GAN(gan, data_pdf=None):
    size = 500
    x = np.linspace(-6,6,100)
    bins = np.linspace(-6,6,60)
    real_data = gan.data_fn(size)
    noise = gan.noise_fn(size)
    sampled_data = gan.generate_samples(noise)
    
    plt.hist(noise.numpy(), label='noise', alpha=0.5, density=True, color='b', bins=bins)
    plt.hist(real_data.numpy(), label='real data', alpha=0.5, density=True, color='g', bins=bins)
    plt.hist(sampled_data.numpy(), label='G samples', alpha=0.5, density=True, color='r', bins=bins)
    
    if data_pdf is not None:
        plt.plot(x, data_pdf(x), 'g', label='real distibution')
    with torch.no_grad():
        plt.plot(x, gan.D(torch.from_numpy(x).float().unsqueeze(-1)).numpy(), 'b', label='D distibution')
    
    plt.legend(loc='upper left')
    plt.show()

In [17]:
visualize_GAN(gan, data_pdf=data_pdf)

In [81]:
mu = 2
noise_fn = lambda x: torch.rand((x, 1), device='cpu')-2
data_fn = lambda x: mu+torch.randn((x, 1), device='cpu')
data_pdf = lambda X: norm.pdf(X-mu)

gen_hiddens = [1,64,64,1]
dis_hiddens = [1,64,64,1]
G = get_simple_model(gen_hiddens)
D = nn.Sequential(*get_simple_model(dis_hiddens), nn.Sigmoid())

gan = VanillaGAN(G, D, noise_fn, data_fn, device='cpu')

In [18]:
plot_gan_data(data_fn, noise_fn, data_pdf)

In [19]:
visualize_GAN(gan, data_pdf=data_pdf)

In [97]:
epochs = 50
batches = 100

In [84]:
step_size = 1
loss_g, loss_d_real, loss_d_fake = [], [], []
start = time()
for epoch in range(epochs):
    break
    loss_g_running, loss_d_real_running, loss_d_fake_running = 0, 0, 0
    for i,batch in enumerate(range(batches)):
        #lg_, (ldr_, ldf_) = gan.train_step()
        ldr_, ldf_ = gan.train_step_D()
        if i%step_size == 0:
            print(i)
            print('D train step')
            visualize_GAN(gan)
        lg_ = gan.train_step_G()
        if i%step_size == 0:
            print('G train step')
            visualize_GAN(gan)
        
        loss_g_running += lg_
        loss_d_real_running += ldr_
        loss_d_fake_running += ldf_
    loss_g.append(loss_g_running / batches)
    loss_d_real.append(loss_d_real_running / batches)
    loss_d_fake.append(loss_d_fake_running / batches)
    print(f"Epoch {epoch+1}/{epochs} ({int(time() - start)}s):"
          f" G={loss_g[-1]:.3f},"
          f" Dr={loss_d_real[-1]:.3f},"
          f" Df={loss_d_fake[-1]:.3f}")
    visualize_GAN(gan, data_pdf=data_pdf)

In [ ]:
######

In [87]:
mu1 = 3
mu2 = -3
k = 0.7

noise_fn = lambda x: torch.rand((x, 1), device='cpu') - 0.5

def data_fn(x):
    a = mu1 + torch.randn((x, 1), device='cpu')
    b = mu2 + torch.randn((x, 1), device='cpu')
    mask = np.random.rand(x) < k
    mask = mask[:, None]
    samples = (a * mask + b * (1 - mask))
    
    return samples.float()

data_pdf = lambda X: k*norm.pdf(X-mu1)+(1-k)*norm.pdf(X-mu2)

In [20]:
plot_gan_data(data_fn, noise_fn, data_pdf)

In [91]:
gen_hiddens = [1,64,64,1]
dis_hiddens = [1,64,64,1]
G = get_simple_model(gen_hiddens)
D = nn.Sequential(*get_simple_model(dis_hiddens), nn.Sigmoid())

gan = VanillaGAN(G, D, noise_fn, data_fn, device='cpu')
loss_g, loss_d_real, loss_d_fake = [], [], []

In [94]:
epochs = 20
batches = 100

In [96]:
start = time()
for epoch in range(epochs):
    #break
    loss_g_running, loss_d_real_running, loss_d_fake_running = 0, 0, 0
    for i,batch in enumerate(range(batches)):
        lg_, (ldr_, ldf_) = gan.train_step()
        loss_g_running += lg_
        loss_d_real_running += ldr_
        loss_d_fake_running += ldf_
    loss_g.append(loss_g_running / batches)
    loss_d_real.append(loss_d_real_running / batches)
    loss_d_fake.append(loss_d_fake_running / batches)
    print(f"Epoch {epoch+1}/{epochs} ({int(time() - start)}s):"
          f" G={loss_g[-1]:.3f},"
          f" Dr={loss_d_real[-1]:.3f},"
          f" Df={loss_d_fake[-1]:.3f}")
    visualize_GAN(gan, data_pdf=data_pdf)